# **I.  Split Very Large Images in Smaller Frames Using an Overlapping Tile Method**

# **II.  Recombine Tiles Following Processing**

NOTE:

A. Divided frames should have shape divisible by the step size.

B. Large original image should have dimensions also divisible by step size and frame dimensions.



- - - 
***Connecting to Local Runtime:**

Enter the following into command prompt:

*jupyter notebook   --NotebookApp.allow_origin='https://colab.research.google.com'   --port=8888   --NotebookApp.port_retries=0 --allow-root --no-browser*

Copy the http address that appears in the **command window**, near the bottom of the current output. (e.g. *http://localhost:8888/?token=a887174a56c905d5421fc88b2086782d53dfa034a7e690d0*) [Note: do NOT copy this address, it's just an example] [Note2: better to use the "localhost:8888" url rather than the "127.0.0.1:8888".  You'll see what I mean]

Connect to Local Runtime by clicking the down arrow next to the "Connect" button in the top right of this window.  Then click "Connect to local runtime".

Paste the copied http address into the input line on the popup window. Click Connect.  (Note: if an old address is already in the line, replace it with the new one)

For more info:
see: https://research.google.com/colaboratory/local-runtimes.html

---

In [ ]:
import numpy as np
import cv2
import os 
import PIL.Image
import skimage.io as ska
from os.path import splitext

PIL.Image.MAX_IMAGE_PIXELS = 933120000

#@markdown ___
#@markdown ### **I.  Divide Your Images** 
#@markdown (Use Tiling to Split Very Large Images into Multiple Smaller Images)

#@markdown ___
#@markdown NOTE: *Recommend dividing into 5120x5120 with 2560 step size, to ensure compatibility woth existing weights.*

#@markdown Set Image Tile Dimensions (MxN) and Set Size for Overlap (S):
M = 5120 #@param {type:"integer"}
N = 5120 #@param {type:"integer"}
S = 2560 #@param {type:"integer"}

#@markdown Resize Output Tile (m x n)?
RE = True #@param{type:"boolean"}
m = 512 #@param {type:"integer"}
n = 512 #@param {type:"integer"}

#@markdown ___
#@markdown Path to Large Image(s):
image_path = 'D:\\Images\\Large' #@param {type:"string"}
#@markdown Save Path for Divided Images:
save_path = 'D:\\Images\\Large\\New' #@param {type:"string"}


def bin_ndarray(ndarray, new_shape, operation='sum'):

  operation = operation.lower()
  if not operation in ['sum', 'mean']:
      raise ValueError("Operation not supported.")
  if ndarray.ndim != len(new_shape):
      raise ValueError("Shape mismatch: {} -> {}".format(ndarray.shape,
                                                          new_shape))
  compression_pairs = [(d, c//d) for d,c in zip(new_shape,
                                                ndarray.shape)]
  flattened = [l for p in compression_pairs for l in p]
  ndarray = ndarray.reshape(flattened)
  for i in range(len(new_shape)):
      op = getattr(ndarray, operation)
      ndarray = op(-1*(i+1))
  return ndarray

for name in os.listdir(image_path):
  impath = os.path.join(image_path,name)
  if os.path.isdir(impath):
    continue

  fname, ext = splitext(name)
  print("Processing: " + "  " + name)
  ll = len(name)
  img = ska.imread(impath)

  if not os.path.exists(os.path.join(save_path, fname)):
    os.makedirs(os.path.join(save_path, fname))

  Holder = np.zeros((M, N), dtype=np.uint8)
  sh = img.shape
  X = sh[0]
  Y = sh[1]

  i = 0
  for x in range(0,(X-M+S),S):
    for y in range(0,(Y-N+S),S):

      savePath = os.path.join(save_path, fname, fname + '_' + str(i) + '.png')
      Holder = img[x:x+M, y:y+N]
      Holder = Holder.astype(np.uint8)
      #print(x, y)
      #re = cv2.cvtColor(Holder.astype(np.float32), cv2.COLOR_BGR2GRAY)
      if RE:
        re = cv2.resize(Holder, (m, n))
        #re = bin_ndarray(Holder, (512, 512), operation='mean')
      else:
        re = Holder
      ska.imsave(savePath, re)
      i = i+1
print("Fin")

In [ ]:
import numpy as np
import cv2
import os 
from PIL import Image
import skimage.io as ska
from skimage.measure import label, regionprops
from skimage.segmentation import flood, flood_fill, clear_border
from scipy import ndimage
import re
import matplotlib.pyplot as plt

Image.MAX_IMAGE_PIXELS = 933120000

#@markdown ___
#@markdown ### **II.  Unite Your Images** 
#@markdown (Use Tiling to Merge Multiple Smaller Images into Very Large Images)

#@markdown (Designed for remerging predictions of the divided images, created in the above section)

#@markdown ___
#@markdown NOTE: *Predictions will be resized to match the Divided images used, however this may or may not be absolutely nessecary, if you are feeling computationally daring.*

#@markdown Set Image Tile Dimensions (MxN) ans Setp Size for Overlap (S):
M = 5120 #@param {type:"integer"}
N = 5120 #@param {type:"integer"}
S = 2560 #@param {type:"integer"}

#@markdown Set United Image Dimensions (Size of Output Image (OxP) following Unification of Tiles)
O = 20480 #@param {type:"integer"}
P = 20480 #@param {type:"integer"}

#@markdown ___
#@markdown Path to Divided Predictions:
image_path = 'D:\\Images\\Large\\00' #@param {type:"string"}
#@markdown Save Path for United Image
save_path = 'D:\\Images\\Large\\00' #@param {type:"string"}

if not os.path.exists(save_path):
    os.makedirs(save_path)

def atoi(text):
    return int(text) if text.isdigit() else text
def natural_keys(text):
    return [ atoi(c) for c in re.split('(\d+)',text) ]

# function to get unique values 
def unique(list1): 
  
    # intilize a null list 
    unique_list = [] 
    listF = list1.flatten()
    # traverse for all elements 
    for x in listF: 
        # check if exists in unique_list or not 
        if x not in unique_list: 
            unique_list.append(x) 
    return unique_list

name = os.listdir(image_path)
name.sort(key=natural_keys)

#for num in name:
Holder = np.zeros((O, P), dtype=np.float16)
Output = np.zeros((O, P), dtype=np.float16)
re2 = np.zeros((N, M), dtype=np.uint8)
#fname = os.listdir(os.path.join(image_path,num))
#fname.sort(key=natural_keys)
i = 0
for x in range(0,(O-N+S),S):
  for y in range(0,(P-M+S),S):
  
    print(name[i])
    
    pic = name[i]
    
    img = ska.imread(os.path.join(image_path,pic))
    img = clear_border(img)
    
    #if img.shape[0] != M or img.shape[1] != N
    res = np.asarray(Image.fromarray(img).resize((N,M), Image.NEAREST))
    res = label(res)
    re1 = res
    re1[re1>0] = 1
    kernel = np.ones((35,35),np.uint8)
    re1 = cv2.erode(re1.astype("uint8"), kernel, iterations=1)
    res[re1==0] = 0
    if i < 0:
      reB=res
      reB[res>0] = 1
      re2B=re2
      re2B[re2>0] = 1
      re3 = reB+re2B
      re3[re3<2] = 0
      re3[re3==2] = 1
      propsB = regionprops(re3)
      for props in propsB:
        y0, x0 = props.centroid
        cv2.floodFill(re, None, (int(x0), int(y0)), int(re2[(int(x0), int(y0))]))
    
    Holder[x:x+N, y:y+M] = res

    Output = np.fmax(Output, Holder[:,:])
    re2 = res
    i = i+1

    # Observe Progress (Slower)
    #show = Output.copy()
    #_, ax = plt.subplots(1, figsize=(16, 16))
    #ax.axis('off')
    #plt.imshow(show.astype('uint8'),clim=(0.0, 1.0))
    #plt.show()

ll2 = len(pic)
print("Saved   " + pic[0:ll2-7])
savePath = os.path.join(save_path, pic[0:ll2-7] + '.png')
out = label(Output)
out = ndimage.grey_dilation(out, size=(35,35))
regions = regionprops(out)
for props in regions:
  area = props.area
  lbl = props.label
  if area < 25000:
    #print(area)
    out[out == lbl] = 0

cv2.imwrite(savePath, out.astype(np.uint16))
#ska.imsave(save_path, out)
